In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

In [ ]:
#load the dataframe
df = pd.read_csv('JensenJanes.csv')

In [ ]:
#take the zscore
zscore_df = pd.DataFrame(stats.zscore(df), columns=df.columns, index=range(0,len(df)))

In [ ]:
#display the zscored data with a heatmap (in the data frame), here picking a color scheme that approximates that used
# by Jensen and Janes
zscore_df.style.background_gradient(cmap='inferno')

## Setup the covariance matrix, find the eigenvectors and values 

In [ ]:
S = zscore_df.cov()

In [ ]:
eigVals, eigVec = np.linalg.eig(S)

In [ ]:
#Check eigVals, are they sorted and in which order (ascending/descending) -- here shows it is in descending
# so the 0th eigenvector is PC1. 
eigVals

In [ ]:
#Calculate the scores by matrix multiplication of the zscore data and the eigenvectors
T = np.matmul(zscore_df.values, eigVec)